satellite:
  memory_capacity_gb: 5 #Capacite du satellite
  image_size_per_km2_gb: 0.15 # Taille d'une image par km2
  image_duration_per_km2_sec: 1.5 # Duree de capture d'une image
  max_photo_duration_s: 120 # Duree max de capture d'une image
  simultaneous_tasks: false # Une tache (image) a la fois
  recalibration_time_s: 10 # Temps de recalibrage en secondes
  
requests:
  - location: "Tokyo"
    coordinates: [35.6895, 139.6917]
    priority: 3
    area_size_km2: 100
    time_window_sec: [0-100]
  - location: "Montréal"
    coordinates: [45.5017, -73.5673]
    priority: 2
    area_size_km2: 50
    time_window_sec: [50-120]
  - location: "New-York"
    coordinates: [40.730610, -73.935242]
    priority: 3
    area_size_km2: 60
    time_window_sec: [50-120]
 - location: "Paris"
    coordinates: [48.8566, 2.3522]
    priority: 1
    area_size_km2: 80
    time_window_sec: [100-150]

Voici mes contraintes en gros, j'aimerai tout prendre en compte 
Je souhaite maximiser la priorite tout en minimisant le temps de deplacement du satellite 

Les coordonnees sont cense etre des coordonnees reeles sur la planete, tu as le droit de donner des reeles coordonnees
Cependant, le satellite est en orbite autour de la terre, il doit passer par le max de points en respectant les fenetres temporelle de visibilite de chauqe points
Je veux aussi respecter la capacite memoire du satellite et la duree max de capture d'une image
Aussi une seule image doit etre prise a la fois 
Les prioritees elevees sont  tres importantes

In [ ]:
from ortools.sat.python import cp_model
import math

satellite = {
    "memory_capacity_gb": 5,
    "image_size_per_km2_gb": 0.15,
    "image_duration_per_km2_sec": 1.5,
    "max_photo_duration_s": 120,
    "simultaneous_tasks": False,
    "recalibration_time_s": 30,
}

requests = [
    {"location": "Tokyo", "coordinates": (35.6895, 139.6917), "priority": 3, "area_size_km2": 10, "time_window_sec": (0, 100)},
    {"location": "Paris", "coordinates": (48.8566, 2.3522), "priority": 1, "area_size_km2": 8, "time_window_sec": (100, 150)},
    {"location": "Montréal", "coordinates": (45.5017, -73.5673), "priority": 2, "area_size_km2": 8, "time_window_sec": (50, 120)},
    {"location": "New-York", "coordinates": (40.730610, -73.935242), "priority": 3, "area_size_km2": 6, "time_window_sec": (50, 120)}
]

# Sort by start time window (for print purpose only)
requests = sorted(requests, key=lambda x: x["time_window_sec"][0])

# CP-SAT Model
model = cp_model.CpModel()

num_requests = len(requests)

# Time variables for each request
start_times = [model.NewIntVar(req["time_window_sec"][0], req["time_window_sec"][1], f"start_{i}") for i, req in enumerate(requests)]

# Capture Duration of each request (if max is reached then the max duration of a photo)
capture_durations = [int(min(req["area_size_km2"] * satellite["image_duration_per_km2_sec"], satellite["max_photo_duration_s"]))
                     for req in requests]

# Select only most relevant places
is_selected = [model.NewBoolVar(f"select_{i}") for i in range(num_requests)]

# One task at a time
for i in range(num_requests):
    for j in range(i + 1, num_requests):
        no_overlap = model.NewBoolVar(f"no_overlap_{i}_{j}")
        
        model.Add(start_times[i] + capture_durations[i] + satellite["recalibration_time_s"] <= start_times[j]).OnlyEnforceIf(no_overlap)
        model.Add(start_times[j] + capture_durations[j] + satellite["recalibration_time_s"] <= start_times[i]).OnlyEnforceIf(no_overlap.Not())

        model.AddImplication(is_selected[i], no_overlap)
        model.AddImplication(is_selected[j], no_overlap)

# Stay in time window
for i, req in enumerate(requests):
    time_window_end = req["time_window_sec"][1]
    model.Add(start_times[i] + capture_durations[i] <= time_window_end)
    

# Memory
mem_used = [requests[i]["area_size_km2"] * satellite["image_size_per_km2_gb"] for i in range(num_requests)]
model.Add(sum(mem_used) <= satellite["memory_capacity_gb"])

# Maximizing the priority
priority_score = sum(requests[i]["priority"] * start_times[i] for i in range(num_requests))
model.Maximize(priority_score)

# Solve
solver = cp_model.CpSolver()
status = solver.Solve(model)

if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    prev_time = 0
    for i in range(num_requests):
        print(f"{requests[i]['location']} : Start at {solver.Value(start_times[i])}s, Duration: {capture_durations[i]}s")
        print(f'Recalibration: {satellite['recalibration_time_s']}')
        total_time = solver.Value(start_times[i]) + capture_durations[i] + satellite['recalibration_time_s']
        print(f'End task, total time: {total_time - prev_time}')
        prev_time = total_time
        
else:
    print("No solution found.")

Tokyo : Start at 24s, Duration: 15s
Recalibration: 30
End task, total time: 69
Montréal : Start at 69s, Duration: 12s
Recalibration: 30
End task, total time: 42
New-York : Start at 111s, Duration: 9s
Recalibration: 30
End task, total time: 39
Paris : Start at 150s, Duration: 12s
Recalibration: 30
End task, total time: 42


In [7]:
satellite = {
    "memory_capacity_gb": 5,
    "image_size_per_km2_gb": 0.15,
    "image_duration_per_km2_sec": 1.5,
    "max_photo_duration_s": 120,
    "simultaneous_tasks": False,
    "recalibration_time_s": 10,
}

requests = [
    {"location": "Tokyo", "coordinates": (35.6895, 139.6917), "priority": 3, "area_size_km2": 10, "time_window_sec": (0, 100)},
    {"location": "Montréal", "coordinates": (45.5017, -73.5673), "priority": 2, "area_size_km2": 5, "time_window_sec": (50, 120)},
    {"location": "New-York", "coordinates": (40.730610, -73.935242), "priority": 3, "area_size_km2": 6, "time_window_sec": (50, 120)},
    {"location": "Paris", "coordinates": (48.8566, 2.3522), "priority": 1, "area_size_km2": 8, "time_window_sec": (100, 150)},
]

In [8]:
from ortools.sat.python import cp_model
import math

# Données du problème
satellite = {
    "memory_capacity_gb": 5,
    "image_size_per_km2_gb": 0.15,
    "image_duration_per_km2_sec": 1.5,
    "max_photo_duration_s": 120,
    "simultaneous_tasks": False,
    "recalibration_time_s": 10,
}

requests = [
    {"location": "Tokyo", "coordinates": (35.6895, 139.6917), "priority": 3, "area_size_km2": 100, "time_window_sec": (0, 100)},
    {"location": "Montréal", "coordinates": (45.5017, -73.5673), "priority": 2, "area_size_km2": 50, "time_window_sec": (50, 120)},
    {"location": "New-York", "coordinates": (40.730610, -73.935242), "priority": 3, "area_size_km2": 60, "time_window_sec": (50, 120)},
    {"location": "Paris", "coordinates": (48.8566, 2.3522), "priority": 1, "area_size_km2": 80, "time_window_sec": (100, 150)},
]

# Modèle CP-SAT
model = cp_model.CpModel()

num_requests = len(requests)

# Variables de décision
start_times = [model.NewIntVar(req["time_window_sec"][0], req["time_window_sec"][1], f"start_{i}") for i, req in enumerate(requests)]
capture_durations = [min(req["area_size_km2"] * satellite["image_duration_per_km2_sec"], satellite["max_photo_duration_s"]) for req in requests]
is_selected = [model.NewBoolVar(f"select_{i}") for i in range(num_requests)]

# Contraintes : Une seule tâche à la fois
for i in range(num_requests):
    for j in range(i + 1, num_requests):
        no_overlap = model.NewBoolVar(f"no_overlap_{i}_{j}")
        model.Add(start_times[i] + capture_durations[i] + satellite["recalibration_time_s"] <= start_times[j]).OnlyEnforceIf(no_overlap)
        model.Add(start_times[j] + capture_durations[j] + satellite["recalibration_time_s"] <= start_times[i]).OnlyEnforceIf(no_overlap.Not())
        model.AddImplication(is_selected[i], no_overlap)
        model.AddImplication(is_selected[j], no_overlap)

# Contraintes de fin dans la fenêtre de temps
for i, req in enumerate(requests):
    time_window_end = req["time_window_sec"][1]
    model.Add(start_times[i] + capture_durations[i] <= time_window_end).OnlyEnforceIf(is_selected[i])

# Capacité mémoire
mem_used = [requests[i]["area_size_km2"] * satellite["image_size_per_km2_gb"] for i in range(num_requests)]
model.Add(sum(mem_used[i] * is_selected[i] for i in range(num_requests)) <= satellite["memory_capacity_gb"])

# Maximisation de la priorité
priority_score = sum(requests[i]["priority"] * is_selected[i] for i in range(num_requests))
model.Maximize(priority_score)

# Solveur
solver = cp_model.CpSolver()
status = solver.Solve(model)

if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    for i in range(num_requests):
        if solver.Value(is_selected[i]):
            print(f"{requests[i]['location']} : Start at {solver.Value(start_times[i])}s, Duration: {capture_durations[i]}s")
else:
    print("No feasible solution found.")


TypeError: Linear constraints only accept integer values and coefficients: IntAffine(expr=FloatAffine(expr=start_1(50..120), coeff=1, offset=75), coeff=1, offset=10) and start_0(0..100)

In [27]:
from ortools.sat.python import cp_model
import math

satellite = {
    "memory_capacity_gb": 5,
    "image_size_per_km2_gb": 0.15,
    "image_duration_per_km2_sec": 1.5,
    "max_photo_duration_s": 120,
    "simultaneous_tasks": False,
    "recalibration_time_s": 30,
}

requests = [
    {"location": "Tokyo", "coordinates": (35.6895, 139.6917), "priority": 3, "area_size_km2": 10, "time_window_sec": (0, 100)},
    {"location": "Paris", "coordinates": (48.8566, 2.3522), "priority": 1, "area_size_km2": 8, "time_window_sec": (100, 150)},
    {"location": "Montréal", "coordinates": (45.5017, -73.5673), "priority": 2, "area_size_km2": 8, "time_window_sec": (50, 120)},
    {"location": "New-York", "coordinates": (40.730610, -73.935242), "priority": 3, "area_size_km2": 6, "time_window_sec": (50, 120)}
]

# Sort by start time window (for print purpose only)
requests = sorted(requests, key=lambda x: x["time_window_sec"][0])

# CP-SAT Model
model = cp_model.CpModel()

num_requests = len(requests)

# Time variables for each request
start_times = [model.NewIntVar(req["time_window_sec"][0], req["time_window_sec"][1], f"start_{i}") for i, req in enumerate(requests)]

# Capture Duration of each request (if max is reached then the max duration of a photo)
capture_durations = [int(min(req["area_size_km2"] * satellite["image_duration_per_km2_sec"], satellite["max_photo_duration_s"]))
                     for req in requests]

# Select only most relevant places
is_selected = [model.NewBoolVar(f"select_{i}") for i in range(num_requests)]

# One task at a time
for i in range(num_requests):
    for j in range(i + 1, num_requests):
        no_overlap = model.NewBoolVar(f"no_overlap_{i}_{j}")
        
        model.Add(start_times[i] + capture_durations[i] + satellite["recalibration_time_s"] <= start_times[j]).OnlyEnforceIf(no_overlap)
        model.Add(start_times[j] + capture_durations[j] + satellite["recalibration_time_s"] <= start_times[i]).OnlyEnforceIf(no_overlap.Not())

        model.AddImplication(is_selected[i], no_overlap)
        model.AddImplication(is_selected[j], no_overlap)

# Stay in time window
for i, req in enumerate(requests):
    time_window_end = req["time_window_sec"][1]
    model.Add(start_times[i] + capture_durations[i] <= time_window_end).OnlyEnforceIf(is_selected[i])
    

# Memory
mem_used = [requests[i]["area_size_km2"] * satellite["image_size_per_km2_gb"] for i in range(num_requests)]
# model.Add(sum(mem_used) <= satellite["memory_capacity_gb"])
model.Add(sum(mem_used[i] for i in range(num_requests)) <= satellite["memory_capacity_gb"])


# Maximizing the priority
# priority_score = sum(requests[i]["priority"] * start_times[i] for i in range(num_requests))
priority_score = sum(requests[i]["priority"] * is_selected[i] for i in range(num_requests))

model.Maximize(priority_score)

# Solve
solver = cp_model.CpSolver()
status = solver.Solve(model)

if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    prev_time = 0
    for i in range(num_requests):
        if solver.Value(is_selected[i]):
            print(f"{requests[i]['location']} : Start at {solver.Value(start_times[i])}s, Duration: {capture_durations[i]}s")
            print(f'Recalibration: {satellite['recalibration_time_s']}')
            total_time = solver.Value(start_times[i]) + capture_durations[i] + satellite['recalibration_time_s']
            print(f'End task, total time: {total_time - prev_time}')
            prev_time = total_time
        else:
            print(f"{requests[i]['location']}, Priority {requests[i]['priority']}: Not selected")
            
        
else:
    print("No solution found.")

Tokyo : Start at 5s, Duration: 15s
Recalibration: 30
End task, total time: 50
Montréal : Start at 50s, Duration: 12s
Recalibration: 30
End task, total time: 42
New-York : Start at 92s, Duration: 9s
Recalibration: 30
End task, total time: 39
Paris : Start at 138s, Duration: 12s
Recalibration: 30
End task, total time: 49
